# Semantic Similarity

In this notebook we follow a pipeline to extract relevant information from a data dump (generated by GPT).

The **Pipeline** is as follows:
* Read the Data
* Extract the Topics (We work with topics since it is easier to prototype with. Full Sentences can also be used.)
* Pre-Process the Data
* Generate Semantic Embeddings using a SentenceTransformer Model
* Perform K-Means Clustering

The last step, *Perform K-Means Clustering*, itself has a few parts:
* Optimize K for K-Means (this can be hard coded, or, as in the notebook, estimate from intertia)
* If preferred, apply Dimensionality Reduction using TSNE or PCA (this has proved to defeat the point of semantic embeddings to a large extent)
* Retrieve Cluster Assignments in the form of indexes
* Use the indexes to get the real clusters as a List of List of Strings (This is also where lexical cluster refinement has been built into)
* Print Clusters in a cohesive manner as well as the optimizations / datapoint filtering that has been done at multiple stages

In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pylcs
import pandas as pd

In [2]:
# Global variables for the pipeline

red1 = None
red2 = None

In [3]:
# Read Dataset

data_path = '../data/I2_1_1000.txt'

with open(data_path,'r') as txtfile:
    biz_ideas = [line.rstrip('\n') for line in txtfile]

In [4]:
def topic(s1:str):
    
    # Returns the topic extracted from the entire output.
    
    return s1[:s1.find('.')]

In [5]:
def preprocess(data:list, refer=None):
    
    # Applies three proprocessing steps: turns every item to lower case, selects only short strings, and eliminates duplicates
    
    data = [datum.lower() for datum in data]
    processed = {}
    
    if refer:
                
        for i in range(len(data)):
            
            if len(data[i]) < 25:
                processed[data[i]] = refer[i]
        
    else:
        data_processed = [datum for datum in data if len(datum) < 25]
        processed = dict(zip(data_processed, data_processed))
        
    global red1
    red1 = (len(data) - len(processed.keys()))/len(data) * 100
    
    return processed

In [6]:
def overlap(s1:str, s2:str):
    
    # Returns the lexical overlap using longest common subsequence between two strings
    
    if len(s1) > len(s2):
        return pylcs.lcs(s1,s2)/float(len(s1))
    else:
        return pylcs.lcs(s1,s2)/float(len(s2))

In [7]:
# Clustering : KMeans

def kmeans(num_clusters:int, data:list):
    
    from sklearn.cluster import KMeans
        
    clustering_model = KMeans(n_clusters = num_clusters)
    clustering_model.fit(data)
    cluster_assignment = clustering_model.labels_

    return cluster_assignment

In [8]:
def optimize_k(data:list):
    
    # This function is used to automatically determine the optimal value of K for K-Means Clustering
    
    from sklearn.cluster import KMeans
    import math
    
    dists = []
    grads = []
    K = range(1,70)
    iner_drop = 0
    iner_last = 0
    
    for n in K:
        k_model = KMeans(n_clusters = n)
        k_model.fit(data)
        dists.append(k_model.inertia_)
        
    """ # This was an experiment to try stopping the optimization early to save time based on KMeans Inertia
        if n == 1:
            iner_last = k_model.inertia_
            grads.append(0)
        else:
            iner_drop = iner_last - k_model.inertia_
            grad = iner_drop/iner_last
            grads.append(grad)
            iner_last = k_model.inertia_
            
    for x, (y,z) in enumerate(zip(dists,grads),1):
        print(f'{x}. {y:.2f} {z:.4f}')
    """
    def calc_dist(x1,y1,a,b,c):
        return abs((a*x1 + b*y1 + c))/(math.sqrt(a**2 + b**2))
        
    a = dists[0] - dists[-1]
    b = K[-1] - K[0]
    c1 = K[0] * dists[-1]
    c2 = K[-1] * dists[0]
    c = c1 - c2
        
    dists_line = []

    for k in range(K[-1]):
        dists_line.append(calc_dist(K[k], dists[k], a, b, c))
                
    num_clusters = dists_line.index(max(dists_line))+1
        
    return num_clusters

In [9]:
# Alternative Approach: DBScan

def dbscan(data:list):
    
    from sklearn.cluster import DBSCAN

    db_default = DBSCAN(eps = 0.0375, min_samples = 3).fit(data)
    cluster_assignment = db_default.labels_

    return cluster_assignment

In [10]:
def reduce_dims(data, alg='tsne', num_components=2):
    
    # Used to reduce dimensions using TSNE or PCA algorithms
    
    topics_red = None
    
    if alg == 'tsne':
        
        from sklearn.manifold import TSNE

        topics_red = TSNE(n_components=num_components).fit_transform(data)
        
    elif alg == 'pca':
        
        from sklearn.decomposition import PCA
        topics_red = PCA(n_components=num_components,svd_solver='full').fit_transform(data)
            
    return topics_red

In [11]:
def get_clusters(num_clusters:int, df):
    
    # Prepares and returns clusters as list of lists
    
    clusters = []
    
    for i in range(num_clusters):
        
        clust_sent = np.where(df['Cluster'] == i)
        clust_points = []
        
        for k in clust_sent[0]:
            
            clust_points.append(df['Topic'][k])
            
        clusters.append(clust_points)
    
    return clusters

In [12]:
def refine_clusters(df, num_clusters:int, threshold=0.7):
    
    # Eliminates lexically similar items using Longest Common Subsequence
    
    refined_clusters = pd.DataFrame()
    reductions = []

    for i in range(num_clusters):

        df_cluster = df.where(df['Cluster'] == i+1)

        df_cluster.dropna(subset=['Text'], inplace=True)

        refined_cluster = pd.DataFrame()

        for j in range(df_cluster.shape[0]-1):
            flag = True

            for k in range(j+1,df_cluster.shape[0]):

                overlap_ = overlap(df_cluster.iloc[j]['Topic'],df_cluster.iloc[k]['Topic'])

                if overlap_ > 0.7:
                    #print('Hit')
                    flag = False
                    break

            if flag:
                    refined_cluster = refined_cluster.append(df_cluster.iloc[j])

        if df_cluster.shape[0]:
                reductions.append((df_cluster.shape[0] - refined_cluster.shape[0]) / df_cluster.shape[0])
                refined_clusters = refined_clusters.append(refined_cluster)

    global red2
    red2 = np.average(np.array(reductions))*100
    
    return refined_clusters

In [13]:
def print_clusters(num_clusters:int, df, n=10):
    
    # Prints clusters in a cohesive manner and also displays how much redundant data has been eliminated

    for i in range(num_clusters):
        
        df_cluster = df.where(df['Cluster'] == i+1)
        df_cluster.dropna(subset=['Text'], inplace=True)
        display(df_cluster.head(n))
        
    global red1, red2
    print(f'\nData trimmed by {red1:.2f}% in preprocessing step, and by {red2:.2f}% in cluster refinement step.\n')

In [14]:
topics_unprocessed = []

for idea in biz_ideas:

    topics_unprocessed.append(topic(idea))
    
ideas = dict(zip())

processed = preprocess(topics_unprocessed,biz_ideas)

df = pd.DataFrame(zip(processed.values(), processed.keys()))
df.columns = ['Text','Topic']

# Create a list of topics from the corpus

topics = list(df['Topic'])

In [15]:
# Generate embeddings using a pre-trained SentenceTransformer model

model = SentenceTransformer('paraphrase-mpnet-base-v2')
topics_embeddings_unprocessed = model.encode(topics_unprocessed)
topics_embeddings = model.encode(topics)

print(f'Shape of topic embeddings before pre-processing: {topics_embeddings_unprocessed.shape}')
print(f'Shape of topic embeddings after pre-processing: {topics_embeddings.shape}')

Shape of topic embeddings before pre-processing: (1000, 768)
Shape of topic embeddings after pre-processing: (487, 768)


In [16]:
# Top K similar ideas (Pre-processed but unrefined)

topic_query = 'Beauty'
query_embedding = model.encode(topic_query)

top_k = 5

cos_scores = util.pytorch_cos_sim(query_embedding, topics_embeddings)[0]
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("Sentence:", topic_query, "\n")
print(f'Top {top_k} most similar items in corpus:\n')

for idx in top_results[0:top_k]:
    print(topics[idx], "(Score: %.4f)" % (cos_scores[idx]))

Sentence: Beauty 

Top 5 most similar items in corpus:

beauty salo (Score: 0.7252)
beauty pageant (Score: 0.5960)
beauty salon (Score: 0.5242)
beauty shop (Score: 0.5123)
a beauty salon (Score: 0.5066)


In [17]:
%%time
# K-Means Pipeline

# Define number of clusters or auto estimate optimum using intertia
num_clusters = optimize_k(topics_embeddings)

# Reduce Dimentions using TSNE or PCA
topics_red = reduce_dims(topics_embeddings,alg='tsne',num_components=2)

# Apply K-Means Clustering
cluster_assignment = kmeans(num_clusters=num_clusters, data=topics_embeddings) # data=topics_embeddings, topics_red
df['Cluster'] = cluster_assignment

# Refine the clusters using LCS
df_refined = refine_clusters(df, num_clusters)
df_refined = df_refined[['Text', 'Topic', 'Cluster']]
df_refined.reset_index(drop=True, inplace=True)

# Print clusters cohesively
print_clusters(num_clusters, df_refined, 5)

# Save data as CSV
df_refined.to_csv(data_path[:-4]+'.csv')

,Text,Topic,Cluster
0,"Florist service. For as low as $1500, one can ...",florist service,1.0
1,Funeral Care Business. One could offer all bas...,funeral care business,1.0
2,Floral Boutique. This is a good option if the ...,floral boutique,1.0
3,Florist. One would have to pay a small salary ...,florist,1.0
4,Floral Design Studio. This business costs less...,floral design studio,1.0


,Text,Topic,Cluster
6,Auto Parts Depot. The prices of all automobile...,auto parts depot,2.0
7,"Auto Shop. One could purchase an auto shop, an...",auto shop,2.0
8,Auto insurance Business. This business can sta...,auto insurance business,2.0
9,Car Shops. One could get all the needed vehicl...,car shops,2.0
10,Auto Insurance. This type of business could ta...,auto insurance,2.0


,Text,Topic,Cluster
22,Bakery. This idea is fairly good. The bakery c...,bakery,3.0
23,Baking Soda Products. For as low as $7500 to s...,baking soda products,3.0
24,Food manufacturing. This option would be profi...,food manufacturing,3.0
25,Small Baking Company. One could purchase ingre...,small baking company,3.0
26,Baked Goods Store. One can set up a small bake...,baked goods store,3.0


,Text,Topic,Cluster
35,Cleaner/Dry cleaner. One could start this as a...,cleaner/dry cleaner,4.0
36,Laundry and Dry Cleaning. This makes about $20...,laundry and dry cleaning,4.0
37,Laundries. One could set up a small laundry bu...,laundries,4.0
38,Laundromat. One could set up a business to pro...,laundromat,4.0
39,Air Conditioning Service. This can also be run...,air conditioning service,4.0


,Text,Topic,Cluster
55,Dental Care. The low interest rate makes this ...,dental care,5.0
56,Dental Practices. Dental offices could be an o...,dental practices,5.0
57,Dental Office. This is an additional product t...,dental office,5.0
58,Dental Shop. One could start a dentistry busin...,dental shop,5.0
59,Beauty Pageant. This option requires less capi...,beauty pageant,5.0


...

,Text,Topic,Cluster
255,Dining Out. One can open a small restaurant wi...,dining out,18.0
256,Food and beverage store. Start with something ...,food and beverage store,18.0
257,Diner/fast food. If one can make a reasonable ...,diner/fast food,18.0
258,"Restaurant services. Again, this option does n...",restaurant services,18.0
259,"Bakery, Cafe, or Deli. One can rent a bakery, ...","bakery, cafe, or deli",18.0


,Text,Topic,Cluster
267,Web Design Company. This one is not the best c...,web design company,19.0
268,Internet Service company. The price is $2000 t...,internet service company,19.0
269,Web Browsers. One could try creating an online...,web browsers,19.0
270,Internet Radio Station. One can set up a radio...,internet radio station,19.0
271,Web Designing. One could set up a company with...,web designing,19.0


,Text,Topic,Cluster
279,Food Truck. One can obtain a food bus for abou...,food truck,20.0
280,"Pizza. At the lower end of the spectrum, getti...",pizza,20.0
281,"Pizza Hut. The same as above, with a price tag...",pizza hut,20.0
282,Pizza restaurant. This is one of the cheapest ...,pizza restaurant,20.0
283,Pizza or Chinese. One could either choose one ...,pizza or chinese,20.0


,Text,Topic,Cluster



Data trimmed by 51.30% in preprocessing step, and by 33.98% in cluster refinement step.

CPU times: user 6min 6s, sys: 5.87 s, total: 6min 12s
Wall time: 45.7 s


In [ ]:
# DBScan Pipeline

# Define number of clusters to show (note number of clusters is automatically determined)
num_clusters = 75

# Reduce Dimentions using TSNE
topics_red = reduce_dims(topics_embeddings,alg='tsne',num_components=3)

# Apply DBScan Clustering
cluster_assignment = dbscan(data=topics_embeddings) # data=topics_embeddings, topics_red
df['Cluster'] = cluster_assignment

# Refine the clusters using LCS
df_refined = refine_clusters(df)
df_refined = df_refined[['Text', 'Topic', 'Cluster']]

# Print clusters cohesively
print_clusters(num_clusters, df_refined, 5)